Student's Name and Email Address

Boise State University, Department of Chemistry and Biochemistry

## CHEM 324: PChem Lab {-}
# Worksheet 3: Bomb Calorimetry {-}

In [ ]:
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
# Load the google drive with your files 
from google.colab import drive
drive.mount('/content/drive')
# The following needs to be the path of the folder with all your collected data in .csv format
path='/content/drive/MyDrive/Colab Notebooks/Bomb_Data/'

In [ ]:
# The following functions have been designed to help you fit the calorimetry data
#
def fit_bomb_data(data,time_start,time_post,time_exp,timestep):
    """
    Function to fit calorimetry curves with three regimes

    Input variables:
        data = a DataFrame with a 'time' and the SECOND column with temperatures
        time_start = skip the first part of the curve
        time_post = arbitrary time after the ignition
        time_exp = reasonable time to perform the fit of the exponential part
        timestep = time interval between collected datapoints

    Output:
        deltaT_corrected = the temperature jump at the middle of combustion
    """
    time_pre = find_preignition_time(data,time_start,time_post,timestep)
    preignition_lr = LinearRegression()
    preignition_data = data.query('time > {} and time < {}'.format(time_start,time_pre))
    x = preignition_data['time'].values.reshape(-1,1)
    y = preignition_data.iloc[:,1].values
    preignition_lr.fit(x,y)
    temperature_preignition = preignition_lr.predict(np.array([[time_pre]]))[0]
    
    postignition_lr = LinearRegression()
    postignition_data=data.query('time > {} '.format(time_post)).dropna()
    x = postignition_data['time'].values.reshape(-1,1)
    y = postignition_data.iloc[:,1].values
    postignition_lr.fit(x,y)
    temperature_postignition = postignition_lr.predict(np.array([[time_post]]))[0]
    deltaT = temperature_postignition - temperature_preignition

    time_det= time_pre+1/rate
    temperature_preignition_corrected = preignition_lr.predict(np.array([[time_det]]))[0]
    temperature_postignition_corrected = postignition_lr.predict(np.array([[time_det]]))[0]
    deltaT_corrected = temperature_postignition_corrected - temperature_preignition_corrected
    return deltaT_corrected

def find_preignition_time(data,time_start,time_post,timestep,threshold=0.8):
    """
    Function to identify the pre-ignition time from calorimetry curves

    Input variables:
        data = a DataFrame with a 'time' and the SECOND column with temperatures
        time_start = skip the first part of the curve
        time_post = arbitrary time after the ignition
        timestep = time interval between collected datapoints
        threshold = the score at which we decide the curve deviates from linear (optional)

    Output:
        preignition_time = the time at which the curve deviates from a linear drift
    """
    times=np.arange(time_start+5*timestep,time_post,timestep)
    preignition_lr=LinearRegression()
    for time in times:
        preignition_data=data.query('time > {} and time < {}'.format(time_start,time))
        x=preignition_data['time'].values.reshape(-1,1)
        y=preignition_data.iloc[:,1].values
        preignition_lr.fit(x,y)
        if preignition_lr.score(x,y) < threshold : 
            preignition_time = time - 10.
            break
    return preignition_time

## Task 1: Calibration {-}

In the lab you performed a series of experiments with a known compound for which you know the enthalpy of combustion. You should have analyzed at least three samples of the benzoic acid standard (abbreviated to BA in the following) to determine the standard deviation in your calorimeter constant.

* Fit the calorimetry curves for the calibration runs and collect the corresponding temperature jumps

In [ ]:
# The following commands will read all the .csv files related to the
# calibration runs and will create a DataFrame with all the calorimetry curves 
# for the different runs
#
calibration_files=['BA1.csv','BA2.csv','BA3.csv'] # make sure to put the names of your actual files
# 
# this loop will find the longest time in the different runs
#
maxtime=0
for file in calibration_files : 
    data=pd.read_csv(path+file,usecols=(1,2),names=['temperature','time']).dropna()
    time = float(data.loc[len(data.index)-1,'time'])
    if time > maxtime : maxtime = time
# 
# now we create the DataFrame with all the experiments
#
calibration_data=pd.DataFrame({'time':np.arange(0.,maxtime,10.)})
for index,file in enumerate(calibration_files) : 
    calibration_data['run'+str(index)]=pd.read_csv(path+file,usecols=(1,2),names=['temperature','time'])['temperature']

In [ ]:
#
# you can now get the temperature jump of one experiment by using the custom-made fit_bomb_data() function
#
detaT1 = fit_bomb_data(calibration_data[['time','run0']],150,800,600,10)

YOUR COMMENTS HERE

* Use the measured weights and the molar weight of BA to convert the temperature jumps into estimated calorimeter constants. Use error propagation to estimate the error in each computed constant. You can use the NIST value of $-3228. \pm 4$ kJ/mol for the enthalpy of combustion value 

In [ ]:
# 
# 
DeltaH_BA = -3228
DeltaH_error = 4

YOUR COMMENTS HERE

* Determine the best estimate of the calorimeter constant, with error using t-stats, for your instrument.

In [ ]:
C=0.
C_error=0.
print("The best estimate for the calorimeter constant is {}".format(C))
print("The standard error associated with this estimate is {}".format(C_error))

YOUR COMMENTS HERE

## Task 2: Enthalpy of Combustion of 1,4-Cyclohexane Dicarboxylic Acid {-}

Your second set of experiments was aimed at collecting the enthalpy of combustion of 1,4-Cyclohexane Dicarboxylic Acid (abbreviated to CHex in the following). 

* Fit the calorimetry curves to determine the temperature jumps in the experiments.
* Using the weights of your samples and the molar weight of CHex, together with the calorimetry constant in the previous step, convert your measures into changes of internal energy.

In [ ]:
# you can copy the commands from above

YOUR COMMENTS HERE

* Convert the internal energy changes into enthalpies and report the best estimate of the combustion enthalpy at $25^{\circ}$ C for CHex, together with its associated error using t-statistics.
* Compare your result with the literature values

In [ ]:
DeltaH_CHex = 0.
DeltaH_CHex_error = 0.
print("The best estimate for the enthalpy of combustion of CHex is {}".format(DeltaH_CHex))
print("The standard error associated with this estimate is {}".format(DeltaH_CHex_error))

YOUR COMMENTS HERE

## Task 3: Enthalpy of Combustion of Cyclopropane Carboxylic Acid {-}

Your last set of experiments was aimed at collecting the enthalpy of combustion of Cyclopropane Carboxylic Acid (abbreviated to CPro in the following). 

* Fit the calorimetry curves to determine the temperature jumps in the experiments.
* Using the weights of your samples and the molar weight of CPro, together with the calorimetry constant in the previous step, convert your measures into changes of internal energy.

In [ ]:
# you can copy the commands from above

YOUR COMMENTS HERE

* Convert the internal energy changes into enthalpies and report the best estimate of the combustion enthalpy at $25^{\circ}$ C for CPro, together with its associated error using t-statistics.
* Compare your result with the literature values

In [ ]:
DeltaH_CPro = 0.
DeltaH_CPro_error = 0.
print("The best estimate for the enthalpy of combustion of CPro is {}".format(DeltaH_CPro))
print("The standard error associated with this estimate is {}".format(DeltaH_CPro_error))

YOUR COMMENTS HERE

## Task 4: Ring Strain Energy {-}

By comparing the two sets of experiments you should be able to estimate how much energy is involved in the formation of small rings. The following questions will only require you to compare results from the steps above.

* Report a graph for at least one combustion run for both CHex and CPro. These should be placed in the same graph with axes properly labeled. 

In [ ]:
# Be creative in your graphs! 

* Compute your best estimate of the ring strain energy and its associated error.
* Compare your value to the same results in the literature with percent error.

In [ ]:
RingStrain = 0.
RingStrain_error = 0.
print("The best estimate for the ring strain energy is {}".format(RingStrain))
print("The standard error associated with this estimate is {}".format(RingStrain_error))

YOUR COMMENTS HERE

In [ ]:
# This cell is used to allow Google Colab to install the tools to convert the notebook to a pdf file
# Un-comment the following lines when you are ready to export the pdf 
#!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
#!pip install pypandoc

In [ ]:
# Use this command to convert the finished worksheet into a pdf 
# NOTE : you may want to change the path of the file, if you are working in a different folder of the Google Drive
#!jupyter nbconvert --no-input --to PDF "/content/drive/MyDrive/Colab Notebooks/Bomb_Worksheet.ipynb"